In [1]:
import yfinance as yf
import pandas as pd
import Black_Scholes
import numpy as np
from datetime import datetime

# Estimate risk free interest using US Treasurey bill yields
t_bill = yf.Ticker('^IRX')
t_bill_data = t_bill.history(period='3mo')
r = np.mean(t_bill_data['Close'])/100
print(f'Estimate for risk free interest rate, r: {r}')

# Estimate realised volatility of Apple stocks
aapl = yf.Ticker('AAPL')
aapl_data = aapl.history(period='3mo')['Close']
realised_vol = Black_Scholes.realised_volatility(aapl_data.values, 1/365.25)
S = aapl_data.values[-1]
print(f'Estimate for realised volatility: {realised_vol}')
print(f'Current Apple stock price:{S}')

# Gather options data

# Times to expiry  
expirations = aapl.options
expiry_times = []
for date in expirations:
    tau = ((pd.to_datetime(date) - datetime.now()).days)/365.25
    if 0.05 < tau < 0.5:
        expiry_times.append(tau) 

# Strike Prices and Market option price
options_data = []
for i in range(len(expiry_times)):
    call_options = aapl.option_chain(expirations[i]).calls
    put_options = aapl.option_chain(expirations[i]).puts
    
    for j in range(len(call_options)):
        market_price = (call_options.iloc[j]['bid'] + call_options.iloc[j]['ask']) / 2
        if abs(S - call_options.iloc[j]['strike'])/S < 0.2 and market_price > (S - call_options.iloc[j]['strike']):
            option = {}
            option['option_type'] = 'c'
            option['expiry_time'] = expiry_times[i]
            option['strike_price'] = call_options.iloc[j]['strike']
            option['market_price'] = call_options.iloc[j]['ask']
            options_data.append(option)

    for j in range(len(put_options)):
        market_price = (put_options.iloc[j]['bid'] + put_options.iloc[j]['ask']) / 2
        if abs(S - put_options.iloc[j]['strike'])/S < 0.2 and market_price > (put_options.iloc[j]['strike'] - S):
            option = {}
            option['option_type'] = 'p'
            option['expiry_time'] = expiry_times[i]
            option['strike_price'] = put_options.iloc[j]['strike']
            option['market_price'] = market_price 
            options_data.append(option)

options_df = pd.DataFrame(options_data)
options_df.to_csv('options_data.csv', index=False)

print('-'*100)
print(options_df.head())


Estimate for risk free interest rate, r: 0.049449999882624704
Estimate for realised volatility: 0.2746527629416358
Current Apple stock price:226.8000030517578
----------------------------------------------------------------------------------------------------
  option_type  expiry_time  strike_price  market_price
0           c     0.054757         190.0         37.65
1           c     0.054757         197.5         30.45
2           c     0.054757         200.0         28.00
3           c     0.054757         202.5         25.30
4           c     0.054757         210.0         17.55
